In [18]:
import sys
print(sys.executable)

/Users/johnboesen/Documents/Code/#Work/tcellmatch/tcell-env/bin/python


In [21]:
# !! Imports aren't working...
import torch
import torch.nn as nn
import math

# Layer Aa Embedding

In [22]:
class LayerAaEmbedding(nn.Module):
    """ A layer class that implements amino acid embedding.

    Instances of this class can be used as layers in the context of tensorflow Models.
    This layer implements 1x1 convolutions to map a given amino acid embedding (such as one-hot) into a learnable
    new space of choosable dimensionality.

    """
    sublayer_conv2d: torch.nn.Conv2d
    fwd_pass: list

    def __init__(
            self,
            shape_embedding: int,
            squeeze_2D_sequence: bool,
            trainable: bool = True,
            dropout: float = 0.1,
            input_shape=None,
            # dtype=tf.float32
            dtype=torch.float32
    ):
        super().__init__()
        if shape_embedding < 0:
            raise ValueError("aa_embedding_dim has to be >0")
        self.dropout = dropout
        # self.input_shapes = input_shape
        self.sublayer_conv2d = None
        self.shape_embedding = shape_embedding if shape_embedding != 0 else input_shape[-1]
        # !! Check in and out # channels
        self.sublayer_conv2d = torch.nn.Conv1d(
            # in_channels=input_shape[-1],
            # out_channels=self.shape_embedding,
            in_channels=26,
            out_channels=20,
            kernel_size=1,
            stride=1,
            padding=0,
            dilation=1,
            groups=1,
            bias=True,
            padding_mode='zeros'
        )
    def forward(self, x):
        if self.shape_embedding is not None:
            x = self.sublayer_conv2d(x)
        return x

# Model Sa

In [ ]:
class ModelSa:

    forward_pass: list
    final_dense: list

    def __init__(
            self,
            labels_dim: int,
            input_shapes: tuple,
            attention_size: List[int],
            attention_heads: List[int],
            split: bool,
            residual_connection=False,
            aa_embedding_dim: Union[int, None] = 0,
            depth_final_dense: int = 1,
            out_activation: str = "linear",
            dropout: float = 0.0
    ):
        """ Self-attention-based feed-forward network.

        Build the feed forward network as a tf.keras.Model object.

        :param dropout: drop out rate for lstm.
        :param attention_size: hidden size for attention, could be divided by attention_heads.
        :param attention_heads: number of heads in attention.
        :param residual_connection: apply residual connection or not.
        :param aa_embedding_dim: Dimension of the linear amino acid embedding, ie number of 1x1 convolutional filters.
            This is set to the input dimension if aa_embedding_dim==0.
        :param depth_final_dense: Number of final densely connected layers. They all have labels_dim number of units
            and relu activation functions, apart from the last, which has either linear or sigmoid activation,
            depending on out_probabilities.
        :param out_activation: Identifier of output activation function, this depends on
            assumption on labels and cost function:

            - "linear" for binding strength data measured as counts
            - "sigmoid" for binary binding events with multiple events per cell
            - "softmax" for binary binding events with one event per cell
        """
        self.args = {
            "labels_dim": labels_dim,
            "input_shapes": input_shapes,
            "attention_size": attention_size,
            "attention_heads": attention_heads,
            "residual_connection": residual_connection,
            "aa_embedding_dim": aa_embedding_dim,
            "depth_final_dense": depth_final_dense,
            "out_activation": out_activation,
            "dropout": dropout,
            "split": split
        }
        self.input_shapes = input_shapes
        self.labels_dim = labels_dim
        self.attention_size = attention_size
        self.attention_heads = attention_heads
        self.residual_connection = residual_connection
        self.aa_embedding_dim = aa_embedding_dim
        self.depth_final_dense = depth_final_dense
        self.out_activation = out_activation
        self.dropout = dropout
        self.split = split
        assert not split, "not implemented"
        self.run_eagerly = False
        self.x_len = input_shapes[4]

        # input_tcr = tf.keras.layers.Input(
        #     shape=(input_shapes[0], input_shapes[1], input_shapes[2]),
        # )
        input_tcr = torch.zeroes(input_shapes[0], input_shapes[1], input_shapes[2])
        input_covar = torch.zeroes(input_shapes[3])
        # input_covar = tf.keras.layers.Input(
        #     shape=(input_shapes[3]),
        #     name='input_covar'
        # )

        # Preprocessing:
        x = input_tcr
        # x = tf.squeeze(x, axis=[1])  # squeeze out chain
        x = torch.squeeze(x, dim=1)  # squeeze out chain
        x = 2 * (x - 0.5)
        # Optional amino acid embedding:
        if aa_embedding_dim is not None:
            x = LayerAaEmbedding(
                shape_embedding=self.aa_embedding_dim,
                squeeze_2D_sequence=True
            )(x)
        # Self-attention layers.
        for i, (w, n) in enumerate(zip(self.attention_size, self.attention_heads)):
            x = LayerMultiheadSelfAttention(
                width_embedding=w,
                n_heads=n,
                residual_connection=self.residual_connection,
                attention_dropout=self.dropout,
                name="sa_" + str(i)
            )(x)
        x = tf.reshape(x, [-1, x.shape[1] * x.shape[2]])
        # Optional concatenation of non-sequence covariates.
        if input_covar.shape[1] > 0:
            x = tf.concat([x, input_covar], axis=1)
        # Final dense layers.from
        self.final_dense = []
        for i in range(self.depth_final_dense):
            x = tf.keras.layers.Dense(
                units=self.labels_dim,
                activation="relu" if i < self.depth_final_dense - 1 else self.out_activation.lower(),
                use_bias=True,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                kernel_regularizer=None,
                bias_regularizer=None,
                activity_regularizer=None
            )(x)
        output = x

        self.training_model = tf.keras.models.Model(
            inputs=[input_tcr, input_covar],
            outputs=output,
            name='model_sa'
        )

## Multihead self attention

In [ ]:
class LayerMultiheadSelfAttention(nn.Module):
    """ Custom Linear layer but mimics a standard linear layer """
    def __init__(self, size_in, size_out):
        super().__init__()
        self.size_in, self.size_out = size_in, size_out
        weights = torch.Tensor(size_out, size_in)
        self.weights = nn.Parameter(weights)  # nn.Parameter is a Tensor that's a module parameter.
        bias = torch.Tensor(size_out)
        self.bias = nn.Parameter(bias)

        # initialize weights and biases
        nn.init.kaiming_uniform_(self.weights, a=math.sqrt(5)) # weight init
        fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weights)
        bound = 1 / math.sqrt(fan_in)
        nn.init.uniform_(self.bias, -bound, bound)  # bias init

    def forward(self, x):
        w_times_x= torch.mm(x, self.weights.t())
        return torch.add(w_times_x, self.bias)  # w times x + b
